# We are doing all of the Data Cleaning, Filling, Normalization Here.

In [38]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [5]:
data = pd.read_csv('./Data/ks-projects-201801.csv',index_col=[0])

# Convert Catagorial Data to Numerical

In [39]:
le = LabelEncoder()

In [42]:
le.fit(data["name"])

TypeError: '<' not supported between instances of 'float' and 'str'

# Basic Stat Analysis

In [37]:
data.head(2)

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00


In [28]:
print("Total Number of Projects:", len(data),"\n"
      "Total Number of Successful Projects:", len(data[data.state == "successful"]),"\n"
      "Successful Rate Among All projects:",str(round(len(data[data.state == "successful"])/len(data)* 100,2)) + "%\n"
      "Number of Catagories: Main : ", len(set(data["main_category"])), "Sub catagories:", len(set(data["category"])),"\n"
     )

Total Number of Projects: 378661 
Total Number of Successful Projects: 133956 
Successful Rate Among All projects: 35.38%
Number of Catagories: Main :  15 Sub catagories: 159 



In [33]:
data.describe()

,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08
